# coroutines

In [2]:
#example of coroutine
def tally():
    score = 0
    while True:
        increment = yield score
        score +=  increment
        

In [3]:
white_sox = tally()
blue_jays = tally()
next(white_sox)
next(blue_jays)

0

In [4]:
white_sox.send(2)

2

In [5]:
blue_jays.send(3)

3

In [14]:
white_sox.send(4)

6

In [15]:
blue_jays.send(1)

4

In [16]:
#the function send of the generator object tally assigns the parameter to the left side of the yield statement in the generator.
#It means that the send() function will repalce the yield statement to the parameter passed to send and continues 
# executing the code until another yield is found that in that case rteturns a new value. then it stops as usual.



In [9]:
# an example where the coroutines is usefull
############## IMPORTANT ###################
# this code is not exactly like in the book!
# THE BOOK USES PYTHON 3.4 CO ROUTINES SYNTAX
# THAT DOES NOT WORK IN PYTHON 3.6
# This code has been modified for python 3.6
############################################
"""
See resources:
    video:https://www.youtube.com/watch?v=idLtMISlgy8&index=52&list=WL
    code from video: https://github.com/ajdavis/coroutines-demo
 
    https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/
    for coroutines in python 3.6.
    
    https://pymotw.com/3/asyncio/coroutines.html#starting-a-coroutine
    
"""
## original code:
    
import re
import asyncio
from inspect import iscoroutine, iscoroutinefunction

file = """unrelated log messages
sd 0:0:0:0 Attached Disk Drive 
unrelated log messages
sd 0:0:0:0 (SERIAL=ZZ12345) 
unrelated log messages
sd 0:0:0:0 [sda] Options 
unrelated log messages
XFS ERROR [sda] 
unrelated log messages
sd 2:0:0:1 Attached Disk Drive 
unrelated log messages
sd 2:0:0:1 (SERIAL=ZZ67890) 
unrelated log messages
sd 2:0:0:1 [sdb] Options 
unrelated log messages
sd 3:0:1:8 Attached Disk Drive 
unrelated log messages
sd 3:0:1:8 (SERIAL=WW11111) 
unrelated log messages
sd 3:0:1:8 [sdc] Options 
unrelated log messages
XFS ERROR [sdc] 
unrelated log messages""".split('\n')

def match_regex(data, regex):
    for line in reversed(data):
        match = re.match(regex, line)
        if match:
            print('Antes del yield', regex)
            regex = yield match.groupd()[0]
            print('Despues del yield', regex)

def get_serials(data):
    ERROR_RE = 'XPS ERROR (\[sd[a-z]\])'
    matcher = match_regex(data, ERROR_RE)
    device = next(matcher)
    while True:
        bus = matcher.send(
             '(sd \S+) {}.*'.format(re.escape(device)))
        serial = matcher.send('{} \(SERIAL=([^)]*)\)'.format(bus))
        yield serial
        device = matcher.send(ERROR_RE)
        

for serial_number in get_serials(file):
    print(serial_number)
    

C:\Users\XI316270\Applications\Anaconda3\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: generator 'get_serials' raised StopIteration


In [1]:
# an example where the coroutines is usefull
############## IMPORTANT ###################
# this code is not exactly like in the book!
# THE BOOK USES PYTHON 3.4 CO ROUTINES SYNTAX
# THAT DOES NOT WORK IN PYTHON 3.6
# This code has been modified for python 3.6
############################################
"""
See resources:
    
    #read the official PEP that proposed this crap
    # they recommend use the try except for stopIterator among other things.
    # with the try except this code below works in Pyhon 3.6 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    https://www.python.org/dev/peps/pep-0479/

    video:https://www.youtube.com/watch?v=idLtMISlgy8&index=52&list=WL
    code from video: https://github.com/ajdavis/coroutines-demo
 
    https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/
    for coroutines in python 3.6.
    
    https://pymotw.com/3/asyncio/coroutines.html#starting-a-coroutine
"""
    
file = """unrelated log messages
sd 0:0:0:0 Attached Disk Drive 
unrelated log messages
sd 0:0:0:0 (SERIAL=ZZ12345) 
unrelated log messages
sd 0:0:0:0 [sda] Options 
unrelated log messages
XFS ERROR [sda] 
unrelated log messages
sd 2:0:0:1 Attached Disk Drive 
unrelated log messages
sd 2:0:0:1 (SERIAL=ZZ67890) 
unrelated log messages
sd 2:0:0:1 [sdb] Options 
unrelated log messages
sd 3:0:1:8 Attached Disk Drive 
unrelated log messages
sd 3:0:1:8 (SERIAL=WW11111) 
unrelated log messages
sd 3:0:1:8 [sdc] Options 
unrelated log messages
XFS ERROR [sdc] 
unrelated log messages""".split('\n')

import re

def match_regex(data, regex):
    for line in reversed(data):
        print('regex [{}] line[{}]'.format(regex, line))
        match = re.match(regex, line)
        if match:
            print('line matched!')
            regex = yield match.groups()[0]

def get_serials(data):
    try:    
        ERROR_RE = 'XFS ERROR (\[sd[a-z]\])'
        matcher = match_regex(data, ERROR_RE)
        device = next(matcher)
        while True:
            bus = matcher.send('(sd \S+) {}.*'.format(re.escape(device)))
            serial = matcher.send('{} \(SERIAL=([^)]*)\)'.format(bus)) 
            yield serial
            device = matcher.send(ERROR_RE)
    except StopIteration:
        return



for serial_number in get_serials(file):
    print(serial_number)
    

regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
regex [XFS ERROR (\[sd[a-z]\])] line[XFS ERROR [sdc] ]
line matched!
regex [(sd \S+) \[sdc\].*] line[unrelated log messages]
regex [(sd \S+) \[sdc\].*] line[sd 3:0:1:8 [sdc] Options ]
line matched!
regex [sd 3:0:1:8 \(SERIAL=([^)]*)\)] line[unrelated log messages]
regex [sd 3:0:1:8 \(SERIAL=([^)]*)\)] line[sd 3:0:1:8 (SERIAL=WW11111) ]
line matched!
WW11111
regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
regex [XFS ERROR (\[sd[a-z]\])] line[sd 3:0:1:8 Attached Disk Drive ]
regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
regex [XFS ERROR (\[sd[a-z]\])] line[sd 2:0:0:1 [sdb] Options ]
regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
regex [XFS ERROR (\[sd[a-z]\])] line[sd 2:0:0:1 (SERIAL=ZZ67890) ]
regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
regex [XFS ERROR (\[sd[a-z]\])] line[sd 2:0:0:1 Attached Disk Drive ]
regex [XFS ERROR (\[sd[a-z]\])] line[unrelated log messages]
reg

In [1]:
# testing code from the internet
# testing code from https://pymotw.com/3/asyncio/coroutines.html#starting-a-coroutine

import asyncio


async def outer():
    print('in outer')
    print('waiting for result1')
    result1 = await phase1()
    print('waiting for result2')
    result2 = await phase2(result1)
    return (result1, result2)


async def phase1():
    print('in phase1')
    return 'result1'


async def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    event_loop.close()

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


In [1]:
# testing above source from Internet, but this time using generator
try:
    del event_loop
    del return_value
except NameError:
    pass
    
import asyncio


@asyncio.coroutine
def outer():
    print('in outer')
    print('waiting for result1')
    result1 = yield from phase1()
    print('waiting for result2')
    result2 = yield from phase2(result1)
    return (result1, result2)


@asyncio.coroutine
def phase1():
    print('in phase1')
    return 'result1'


@asyncio.coroutine
def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    event_loop.close()


in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')
